In [ ]:
from finlab import data

#guide can be found here
#https://mrjbq7.github.io/ta-lib/funcs.html
#https://twopirllc.github.io/pandas-ta/#indicators-by-category
#pip install --upgrade numpy
#!rew install ta-lib https://blog.quantinsti.com/install-ta-lib-python/#windows
#pip install pandas-ta lib
from finlab import data
import finlab
from finlab.backtest import sim
from finlab import backtest
finlab.login(api_token="QNBAVxVcsFjg0vZ5Um8P4XVtRWnhnMtznBpGjGHXqQ+vBYWkN6kXnXDrW70KhH4v#free")


In [ ]:
#策略開發10分鐘上手 test
from finlab import data
from finlab.backtest import sim
close = data.get('price:收盤價')
pb = data.get('price_earning_ratio:股價淨值比')  
f = data.get("institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)")
i = data.get("institutional_investors_trading_summary:投信買賣超股數")
s = data.get("institutional_investors_trading_summary:自營商買賣超股數(自行買賣)")
董監持有股數占比 = data.get("internal_equity_changes:董監持有股數占比")
boss_hold_rise = 董監持有股數占比.rise(1)
sma20 = close.average(7)
sma60 = close.average(60)
entries = (close > sma20) & (f>0) & (i>0) & (s>0)#&boss_hold_rise
exits = close < sma60
position = entries.hold_until(exits, rank=-pb)
backtest_report = sim(position,resample="W")

In [ ]:
# 收盤價剛站上布林通道上緣 tested
df=data.get('price:收盤價')
mean_20=df.rolling(20).mean()
std_value=df.rolling(20).std()
up=mean_20+std_value*2
cond1=(df>up)&(df.shift()<up)
cond2=(df>5)&(df<200)
position=(cond1&cond2)
position=position[position>0]
#position=position.is_smallest(20)
sim(position,resample="W")

In [ ]:
#均線多頭 tested
df=data.get('price:收盤價')
cond1=df>df.rolling(5).mean()
cond2=df>df.rolling(10,min_periods=5).mean()
cond3=df>df.rolling(20,min_periods=10).mean()
cond4=df>df.rolling(60,min_periods=40).mean()
cond5=f>0
cond6=i>0
cond7=s>0
position=(df*(cond1&cond2&cond3&cond4&cond5&cond6&cond7))
position1=position[position>0]
#position=position.is_largest(10)
sim(position1,resample="M")

In [ ]:
#股價創新高動能 tested
from finlab.backtest import sim
close = data.get("price:收盤價")
roe = data.get("fundamental_features:ROE稅後")
f = data.get("institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)")
i = data.get("institutional_investors_trading_summary:投信買賣超股數")
s = data.get("institutional_investors_trading_summary:自營商買賣超股數(自行買賣)")

position = (close == close.rolling(250).max())&(roe>0) #&(f>0)&(i>0)&(s>0)

report = sim(position, resample="M")
report.display()

In [ ]:
#高 RSI 技術指標策略 TESTED
from finlab import data
from finlab.backtest import sim
#foreign_buy=data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')
#foreign_cont=foreign_buy>0
#increase=data.get('internal_equity_changes:董監增加股數')
#cond7=increase>0
roe = data.get("fundamental_features:ROE稅後")
# 選出 RSI 最大的 20 檔股票
rsi = data.indicator('RSI')
position = rsi.is_largest(40) & roe>0 
# 回測，每週(W)調整一次
report = sim(position, resample='W')

In [ ]:
#乖離率和財報濾網 tested
from finlab import data
from finlab.backtest import sim
# 下載 ROE 跟收盤價
roe = data.get("fundamental_features:ROE稅後")
close = data.get("price:收盤價")
position = ((close / close.shift(60)).is_largest(30) & (roe > 0))
# 回測，每月(M)調整一次
report = sim(position, resample="M", upload=False)
report.display()
from finlab import data
from finlab.backtest import sim
# 下載 ROE 跟收盤價
roe = data.get("fundamental_features:ROE稅後")
close = data.get('price:收盤價')
position = (
      (close / close.shift(60)).is_largest(30) # 選出乖離率前 30 名的股票
    & (roe > 0) # 選出 ROE 大於 0 的股票
)

# 回測，每月(M)調整一次
sim(position, resample='W')

In [ ]:
#本益成長比策略 tested
from finlab import data
from finlab.backtest import sim
rev = data.get('monthly_revenue:當月營收')
rev_ma3 = rev.average(3)
rev_ma12 = rev.average(12)
# 營收趨勢多頭策略
cond1 = rev_ma3/rev_ma12 > 1.1
cond2 = rev/rev.shift(1) > 0.9
cond_all = cond1 & cond2
pe = data.get('price_earning_ratio:本益比')
營業利益成長率 = data.get('fundamental_features:營業利益成長率')
# 本益成長比
peg = (pe/營業利益成長率)
# 本益成長比和原訊號相乘，若不持有則相乘後等於0
position = peg*(cond_all)
# 原訊號為0的不要選，若沒加這行且策略只選到7檔，之後還是會選3檔訊號為0(不持有)的補足10檔，執行這行就會排除訊號為0。
position = position[position>0]
# 選股挑本益成長比前10小的
position = position.is_smallest(10)
sim(position,resample="W")

In [ ]:
#小資族優等生策略 tested
from finlab import data
from finlab.backtest import sim
股本 = data.get('financial_statement:股本')
price = data.get('price:收盤價')
市值 = 股本 * price / 10 * 1000
df1 = data.get('financial_statement:投資活動之淨現金流入_流出')
df2 = data.get('financial_statement:營業活動之淨現金流入_流出')
自由現金流 = (df1 + df2).rolling(4).mean()
稅後淨利 = data.get('fundamental_features:經常稅後淨利')
權益總計 = data.get('financial_statement:股東權益總額')
股東權益報酬率 = 稅後淨利/ 權益總計
營業利益成長率 = data.get('fundamental_features:營業利益成長率')
當月營收 = data.get('monthly_revenue:當月營收')* 1000
當季營收 = 當月營收.rolling(4).sum()
市值營收比 = 市值 / 當季營收
foreign_buy=data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')
foreign_buy_t=foreign_buy.loc[foreign_buy.index[1:]]
foreign_buy_tm1=foreign_buy.shift(1).loc[foreign_buy.index[1:]]
foreign_buy_diff=foreign_buy_tm1-foreign_buy
cond6=foreign_buy>0
increase=data.get('internal_equity_changes:董監增加股數')
cond7=increase>0
rsi = data.indicator('rsi')
rsi_con=rsi>75
condition1 = (市值 >1e9)
condition2 = 自由現金流 > 0
condition3 = 股東權益報酬率 > 0
condition4 = 營業利益成長率 > 0
condition5 = 市值營收比 < 5
position = condition1& condition2 & condition3 & condition4 & condition5 &cond7
report = sim(position, resample='W')
report.display()

In [ ]:
#多空對沖
from finlab import data
from finlab import backtest
close = data.get('price:收盤價')
position = close < 0
position['2330'] = 0.5
position['1101'] = -0.5
report = backtest.sim(position, upload=False)
report.display()
from finlab import data
from finlab import backtest
pb = data.get('price_earning_ratio:股價淨值比')
# 股價淨值比前20%低
position = pb < pb.quantile_row(0.2)
position /= position.sum(axis=1)
# 多單總部位70%
position *= 0.7
# 空單總部位30%
position['0050'] = -0.3
report=backtest.sim( position, upload=False, resample='Q')
report.display()

In [ ]:
#營建產業限定策略 tested
from finlab import data
from finlab.backtest import sim
import pandas as pd
# 限定回測產業範圍
with data.universe(category='建材營造'):
    # 資料引入
    close = data.get('price:收盤價')
    contract_debt = data.get('financial_statement:合約負債_流動')
    contract_debt_gr = (contract_debt/(contract_debt.shift())-1)
    equity = data.get('financial_statement:股本')
    # 計算合約負債佔股本的比率，比率越高代表未來潛在貢獻每股營收越高
    ce_ratio = (contract_debt / equity)
    # 選股條件
    cond1 = ce_ratio > 0.5
    cond2 = (contract_debt_gr > 0.05) & (contract_debt_gr<0.5)
    cond_all = cond1 & cond2
    result = ce_ratio * (cond_all)
    position = result[result > 0].is_largest(5)
    # position_limit 限定單檔標的最多持有30%部位，避免標的數過少時產生重壓的非系統性風險
    report = sim(position,resample='M',position_limit=0.3,stop_loss=0.1,fee_ratio=1.425/1000*0.3)
report.display()

In [ ]:
#營收動能 tested
from finlab import data
from finlab.backtest import sim
import pandas as pd
rev = data.get("monthly_revenue:當月營收")
rev_rf = data.get("monthly_revenue:去年同月增減(%)")
vol = data.get("price:成交股數")/1000
rev_recent_3 = rev.rolling(3).sum()
vol_avg = vol.average(10)
cond1 = (rev_recent_3/rev_recent_3.rolling(24, min_periods=12).max())==1
# 成交量條件，排除流動性過小
cond2 = vol_avg > 1000

稅後淨利率 = data.get("fundamental_features:稅後淨利率")
atpm_trend = (稅後淨利率>3).sustain(1)


ROE綜合損益 = data.get("fundamental_features:ROE綜合損益")
roe_trend = (ROE綜合損益>0).sustain(4)



# 過濾全額交割股
full_cash_delivery_stock_filter = data.get("etl:full_cash_delivery_stock_filter")
cond_all = cond1 & cond2 & full_cash_delivery_stock_filter & atpm_trend &roe_trend
result = rev_rf*(cond_all)
position = result[result>0].is_largest(10).reindex(rev.index_str_to_date().index, method="ffill")
report = sim(position=position,stop_loss=0.3,position_limit=0.1)
report.display()

In [ ]:
#現金流量策略 tested
from finlab import data
from finlab.backtest import sim
營業現金流 = data.get("financial_statement:營業活動之淨現金流入_流出").deadline()
投資現金流 = data.get("financial_statement:投資活動之淨現金流入_流出").deadline()
融資現金流 = data.get("financial_statement:籌資活動之淨現金流入_流出").deadline()
roe = data.get("fundamental_features:ROE稅後").deadline()
close = data.get("price:收盤價")
position = (營業現金流 > 0) & (投資現金流 > 0) & (融資現金流 < 0) & (roe > 5) & (close > 10)
from finlab import data
from finlab.backtest import sim
營業現金流 = data.get("financial_statement:營業活動之淨現金流入_流出").deadline()
投資現金流 = data.get("financial_statement:投資活動之淨現金流入_流出").deadline()
融資現金流 = data.get("financial_statement:籌資活動之淨現金流入_流出").deadline()
roe = data.get("fundamental_features:ROE稅後").deadline()
close = data.get("price:收盤價")
position = (營業現金流 > 0) & (投資現金流 > 0) & (融資現金流 < 0) & (roe > 0) & (close > 10)
# 過濾 KY股
sc = data.get("security_categories")
position_col = position.columns
ky_filter = position_col[~position_col.isin(list(sc[sc["name"].str.contains("KY")]["stock_id"]))]
position = position[ky_filter]
report = sim(position,resample="M",name="現金流正數")
report.display()

In [ ]:
#大盤指標濾網 tested
from finlab import data
from finlab.backtest import sim
import numpy as np
# 市場多空排列家數
def ls_order_position(short=5,mid=10,long=30):
    close = data.get("price:收盤價")
    short_ma = close.average(short)
    mid_ma = close.average(mid)
    long_ma = close.average(long)
    long_order = (short_ma>=mid_ma) & (mid_ma>=long_ma)
    long_order = long_order.sum(1)
    short_order = (short_ma<mid_ma) & (mid_ma<long_ma)
    short_order = short_order.sum(1)
    entry = long_order > short_order
    cond = ~close.isna()
    position = cond & entry
    return position
close = data.get("price:收盤價")
sma20 = close.average(20)
rev = data.get('monthly_revenue:當月營收')
# 多頭排列
cond1 = (close > close.shift(20)) & (close > close.shift(60))
# 短期營收大於長期營收
cond2 = (rev.average(3) > rev.average(12)).sustain(3)
buy = cond1 & cond2
sell = sma20 < sma20.shift()
position = pd.DataFrame(np.nan, index=buy.index, columns=buy.columns)
position[buy] = 1
position[sell] = 0
position = position.ffill().fillna(0).astype(bool)
position = position & ls_order_position()
report = sim(position, resample="D")
report.display()

In [ ]:
#布林通道策略 tested
from finlab import data
from finlab.backtest import sim
close=data.get('price:收盤價')
upperband, middleband, lowerband = data.indicator('BBANDS', resample='D', nbdevup=2.5, nbdevdn=2.5, timeperiod=40)
entries = lowerband > close
exits = close > middleband
position = entries.hold_until(exits,nstocks_limit=100000)['2330']
report=sim(position, upload=True, mae_mfe_window=40)

In [ ]:
#小於六策略 tested
#from finlab import data
from finlab import backtest
# 只買入價格小於 6 元的股票
close = data.get('price:收盤價')
上月比較增減 = data.get("monthly_revenue:上月比較增減(%)")
rev_mm = 上月比較增減 > 30
營業利益率 = data.get("fundamental_features:營業利益率")
opm_trend = (營業利益率>3).sustain(1)

ROA綜合損益 = data.get("fundamental_features:ROA綜合損益")
roa_trend = (ROA綜合損益 > 0).sustain(4)
ROE綜合損益 = data.get("fundamental_features:ROE綜合損益")
roe_trend = (ROE綜合損益>0).sustain(4)


position = (close < 10) & roa_trend & roe_trend
# 回測，每月底(M)重新調整股票權重
report = backtest.sim(position, resample='M')


In [ ]:
#本益成長比策略 tested
from finlab import data
from finlab.backtest import sim
pe = data.get('price_earning_ratio:本益比')
rev=data.get('monthly_revenue:當月營收')
rev_ma3=rev.average(3)
rev_ma12=rev.average(12)
營業利益成長率=data.get('fundamental_features:營業利益成長率').deadline()
peg=(pe/營業利益成長率)
cond1= rev_ma3/rev_ma12>1.1
cond2=rev/rev.shift(1)>0.9
cond_all=cond1 & cond2
result=peg*(cond_all)
position=result[result>0].is_smallest(10).reindex(rev.index_str_to_date().index, method='ffill')
sim(position=position,resample="W",fee_ratio=1.425/1000/3,stop_loss=0.1,name='本益成長比')

In [ ]:
#股價淨值比策略 tested
from finlab import data
from finlab import backtest
收盤價 = data.get('price:收盤價')
股價淨值比 = data.get("price_earning_ratio:股價淨值比")
# 條件 1
收盤價大於五 = 收盤價 > 5
# 條件 2
季線 = 收盤價.rolling(60).mean()
收盤價在季線之上 = 收盤價 > 季線
# 條件 3
總條件 = (收盤價 * 股價淨值比)[收盤價大於五&收盤價在季線之上].is_smallest(20)
backtest.sim(總條件, resample='Q')

In [ ]:
# 營收趨勢多頭策略 teted
from finlab import data
from finlab.backtest import sim
rev = data.get('monthly_revenue:當月營收')
rev_ma3 = rev.average(3)
rev_ma12 = rev.average(12)
cond1 = rev_ma3/rev_ma12 > 1.1
cond2 = rev/rev.shift(1) > 0.9
cond_all = cond1 & cond2
pe = data.get('price_earning_ratio:本益比')
營業利益成長率 = data.get('fundamental_features:營業利益成長率')
# 本益成長比
peg = (pe/營業利益成長率)
# 本益成長比和原訊號相乘，若不持有則相乘後等於0
position = peg*(cond_all)
# 原訊號為0的不要選，若沒加這行且策略只選到7檔，之後還是會選3檔訊號為0(不持有)的補足10檔，執行這行就會排除訊號為0。
position = position[position>0]
# 選股挑本益成長比前10小的
position = position.is_smallest(10)
sim(position)

In [ ]:
#肯特納通道突破策略 tested
from finlab.backtest import sim
from finlab import data
data.set_universe(market='TSE_OTC')
adj_close = data.get('etl:adj_close')
ema_5=data.indicator('EMA', adjust_price=True, resample='D',timeperiod=5)
ema_10=data.indicator('EMA', adjust_price=True, resample='D',timeperiod=10)
ema_20=data.indicator('EMA', adjust_price=True, resample='D',timeperiod=20)
atr=data.indicator('ATR', adjust_price=True, resample='D',timeperiod=10)
keltner_up=ema_10+2*atr
vol=data.get('price:成交股數')/1000
vol_avg=vol.average(20)
cond1 = (adj_close > keltner_up) & (adj_close.shift() < keltner_up.shift())
cond2 = (ema_5 > ema_10) & (ema_10 > ema_20)
cond3 = vol_avg > 100
entries =  cond1 & cond2 & cond3
exits = adj_close < keltner_up
position = entries.hold_until(exits,nstocks_limit=20, rank=vol_avg)
rr=sim(position,resample='W',position_limit=0.05)

In [ ]:
#護國神山救難隊 tested
from finlab import data
from finlab.backtest import sim
close=data.get('price:收盤價')
bbands= data.indicator('BBANDS', resample='D', nbdevup=2.5, nbdevdn=2.5,timeperiod=40)
upperband=bbands[2]
middleband=bbands[1]
lowerband=bbands[0]
price = data.get('price:收盤價')
股本 = data.get('financial_statement:股本')
price = data.get('price:收盤價')
市值 = 股本 * price / 10 * 1000
entries = (lowerband > close) & (市值.is_largest(100)) 
exits = close > middleband
# 設定賣出條件
position = entries.hold_until(exits)
report = sim(position,resample="W")

In [ ]:
#3rsi tested online tested
# 沒有程式碼在雲端喔！
print("hello world")
from finlab import data, backtest
# talib
# real = RSI(close, timeperiod=14)

rsi20 = data.indicator("RSI", timeperiod=20)
rsi60 = data.indicator("RSI", timeperiod=60)
rsi120 = data.indicator("RSI", timeperiod=120)
ROE稅後 = data.get("fundamental_features:ROE稅後")
foreign_buy=data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')	
foreign_cont=(foreign_buy>0).sustain(2)
長週期上漲 = rsi120 > 55
中週期別過熱 = rsi60 < 75
短週期RSI上漲 = rsi20.pct_change(3) > 0.05
短週期RSI高檔頓化 = (rsi20 > 75).sustain(3)
ROE為正 = ROE稅後 > 0
營業利益成長率 = data.get('fundamental_features:營業利益成長率')
cond7=營業利益成長率>0
#股東權益報酬率 = 稅後淨利/ 權益總計

成交股數 = data.get("price:成交股數")
vol_ma = 成交股數.average(5)> 1000000

營業利益成長率 = data.get('fundamental_features:營業利益成長率')
foreign_buy_t=foreign_buy.loc[foreign_buy.index[1:]]
foreign_buy_tm1=foreign_buy.shift(1).loc[foreign_buy.index[1:]]
#foreign_buy_diff=foreign_buy_tm1-foreign_buy
買 = (長週期上漲 
     & 中週期別過熱 
     & 短週期RSI上漲 
     & 短週期RSI高檔頓化
     & ROE為正
     #& foreign_buy_diff
     & cond7
     &vol_ma
    )

持有60天 = 買.shift(60)
收盤價 = data.get('price:收盤價')
跌破季線 = 收盤價 < 收盤價.average(60)

賣 = 持有60天 | 跌破季線 
股票部位 = 買.hold_until(賣)

backtest.sim(股票部位, resample='W')
print("condition ")

In [ ]:
'''
條件一: 公司總市值 < 市場平均總市值

條件二: 過去五年稅後淨利皆為正
data.get('fundamental_features:經常稅後淨利')
條件三: 過去三年稅後淨利成長率皆 >= 15%
data.get('fundamental_features:經常稅後淨利').rise()>0.15.sustain(3) and 

條件四: 預估稅後淨利成長率 >= 15%
data.get('fundamental_features:經常稅後淨利').sustain

條件五: 近五年平均營運現金流量 > 近五年平均稅後淨利
data.get('fundamental_features:每股現金流量')*股數>
條件六: 近年營運現金流量 > 近年稅後淨利
data.get('fundamental_features:每股現金流量')
條件七: 近年營業利益率 >= 10%
data.get('fundamental_features:營業利益率')>10%
條件八: 近年可運用資本報酬率 >= 10%
data.get('fundamental_features:ROA綜合損益')
條件九: 最近一季負債/淨值比 < 50%
data.get('fundamental_features:總負債除總淨值')
條件十: 最近董監事持股比例 >= 20% 或是 最新一期董監事持股比例增加
data.get('internal_equity_changes:董監持有股數占比')
交易面條件

條件十一: 預估本益比 <= 20

條件十二: 預估本益比/預估稅後盈餘成長率比值 <= 1.2

條件十三: 超額報酬(月) > 0
https://medium.com/tej-api-%E9%87%91%E8%9E%8D%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90/%E5%AF%A6%E6%88%B0%E6%87%89%E7%94%A8-%E4%B8%83-%E5%90%89%E5%A7%86-%E5%8F%B2%E8%90%8A%E7%89%B9-jim-slater-%E7%A5%96%E9%AD%AF%E5%8E%9F%E5%89%87%E6%8A%95%E8%B3%87%E6%B3%95-5de2c3c445c5
#實戰應用7
total_value=data.get('dividend_tse:最近一次申報每股 (單位)淨值')*data.get('company_basic_info:已發行普通股數或TDR原發行股數')
#實戰應用(六)
df1=data.get('fundamental_features:ROE稅後')
'''

In [ ]:
''' 說到吉娃娃，你想到什麼呢？

小狗狗最鮮明的性格就是怕生、膽小、愛叫，這支策略充分反映吉娃娃的性格。

不喜歡人多的股票，要有主人(主力) 呵護著。使用集保餘額當作主要選股資料，選散戶較少、大戶持股上升的標的。

很愛叫訊號，一言不合就對主人狂吠，屬於短週期的策略。

很膽小，停損設的小，不妙就趕緊跑走。

但若挖掘吉娃娃的內在，就會發現其實牠對主人超級忠誠，接受一些簡單的訓練與指令，總是會給主人獲利的回報。


動能突破策略。

主要進場條件為營收與股價皆創新高。過濾營收絕對值過小的股票(ex:新藥股)、月均量過小、高價股。每月營收截止日產生換股訊號，最大檔數10檔，每檔最多佔比20%。

大波動策略，停損設定為20%，停利設定為80%。

小蝦米跟大鯨魚
集保餘額與月營收成長綜合選股，找出大戶高持股的穩定成長股。



動能策略，大多頭會打出兇悍的一波流，勝率一半一半，但勝手常持有後一去不回，回檔小。因此停損設8%，偏低位置，保留多數獲利部位與下檔波動，遇到中期回檔也砍得快。

持股用流動性條件篩成5檔，每檔最多持有 33.3% ，避免只選到1-2檔股票時的重壓個股風險。



選股條件

股價創年新高
排除月營收連3月衰退10%以上
排除月營收成長趨勢過老
確認營收底部
排除營收跳動劇烈者
流動性條件



'''

In [ ]:
!pip install ta-lib

In [ ]:
#self
close = data.get("price:收盤價")
sma = 收盤價.average(60)
slowk,slowd = data.indicator("STOCH",adjust_price=False, resample="D", fastk_period=5,slowk_period=3,slowk_matype=0,slowd_period=3,slowd_matype=0)
kd_gold_cross = (slowk > slowd) & (slowk.shift() < slowd.shift())
收盤價 = data.get("price:收盤價")
upperband,middleband,lowerband = data.indicator("BBANDS", timeperiod=10)
cond = (收盤價 < lowerband) & (收盤價.shift() > lowerband.shift())

roe = data.get("fundamental_features:ROE稅後")
pat = data.get("fundamental_features:稅後淨利率")
patg = data.get("fundamental_features:稅後淨利成長率")
f = data.get("institutional_investors_trading_summary:外資自營商買賣超股數")
i = data.get("institutional_investors_trading_summary:投信買賣超股數")
s = data.get("institutional_investors_trading_summary:自營商買進股數(自行買賣)")
pos=(roe>0) &(pat>0)&(patg>0)&((f>0)|(i>0)|(s>0))*(cond)
sim(pos,resample="M")
